##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This is a quick demo of Gemma running on KerasNLP.

Note that you will need a large GPU (e.g. A100) to run this as well.

General Keras reading:
- [Getting started with Keras](https://keras.io/getting_started/)
- [Getting started with KerasNLP](https://keras.io/guides/keras_nlp/getting_started/)

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/Keras_Gemma_2_Quickstart_Chat.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Access

In [1]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".

## Installation

In [ ]:
# Install all deps
!pip install -U keras-nlp
!pip install -U keras==3.3.3

## Quickstart

In [3]:
import keras_nlp
import keras

# Run at half precision.
keras.config.set_floatx("bfloat16")

# using 9B instruction-tuned model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_9b_en")
gemma_lm.summary()

100%|██████████| 780/780 [00:00<00:00, 1.18MB/s]
100%|██████████| 17.2G/17.2G [02:41<00:00, 114MB/s]
100%|██████████| 315/315 [00:00<00:00, 500kB/s]
100%|██████████| 4.04M/4.04M [00:00<00:00, 16.9MB/s]


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 3584)        │   9,241,705,984 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     917,504,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 9,241,705,984 (17.21 GB)

 Trainable params: 9,241,705,984 (17.21 GB)

 Non-trainable params: 0 (0.00 B)

### Define formatting helper functions

In [4]:
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Create a chat helper to manage the conversation state

In [5]:
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with the start turn marker.
      Model will generate end turn marker.
      """
      self.history.append(self.__START_TURN_MODEL__ + message)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=4096)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result


### Chat with the model

In [6]:
chat = ChatState(gemma_lm)
message = "help me understand how to make coffee"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>help me understand how to make coffee</blockquote></font><font size='+1' color='teal'>🤖

> ##  Let's Brew Some Coffee! ☕️
> 
> Making coffee can be as simple or complex as you like. Here's a breakdown of some popular methods, from beginner-friendly to more advanced:
> 
> **1. The Quick & Easy: Instant Coffee**
> 
> * **What you need:** Instant coffee granules, hot water
> * **How to:** Simply add a spoonful of instant coffee to a mug, pour hot water over it, stir, and enjoy!
> 
> **2. The Classic: Drip Coffee Maker**
> 
> * **What you need:** Drip coffee maker, ground coffee, filter
> * **How to:**
>     * Fill the water reservoir with fresh, cold water.
>     * Place a filter in the filter basket.
>     * Add the desired amount of ground coffee to the filter.
>     * Turn on the coffee maker and let it do its thing!
> 
> **3. The French Press:**
> 
> * **What you need:** French press, coarse ground coffee, hot water
> * **How to:**
>     * Add coarse ground coffee to the French press.
>     * Pour hot water (just off the boil) over the coffee grounds.
>     * Stir gently and let it steep for 4 minutes.
>     * Slowly press the plunger down to separate the grounds from the coffee.
>     * Pour and enjoy!
> 
> **4. The Pour Over:**
> 
> * **What you need:** Pour over cone, filter, medium-fine ground coffee, hot water
> * **How to:**
>     * Place a filter in the cone and rinse it with hot water to remove any paper taste.
>     * Add medium-fine ground coffee to the filter.
>     * Slowly pour hot water over the grounds in a circular motion, ensuring all grounds are saturated.
>     * Let the coffee drip through into your cup.
> 
> **5. The Espresso Machine:**
> 
> * **What you need:** Espresso machine, finely ground coffee, tamper
> * **How to:**
>     * Grind your coffee beans finely.
>     * Tamp the coffee grounds firmly into the portafilter.
>     * Lock the portafilter into the espresso machine.
>     * Brew your espresso shot!
> 
> **Tips for Delicious Coffee:**
> 
> * **Use fresh, quality beans:** The better the beans, the better the coffee.
> * **Grind your beans right before brewing:** This preserves freshness and flavor.
> * **Use filtered water:** Impurities in water can affect the taste of your coffee.
> * **Experiment with different brewing methods:** Find what you like best!
> * **Don't be afraid to adjust:** Play with the amount of coffee, water, and brewing time to find your perfect cup.
> 
> 
> Let me know if you have any other questions or want to explore a specific brewing method in more detail!<end_of_turn>

</font>

In [7]:
message = "what is the best coffee?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>what is the best coffee?</blockquote></font><font size='+1' color='teal'>🤖

> Ah, the eternal question! There's no single "best" coffee because taste is subjective. What one person finds delicious, another might find bitter or bland. 
> 
> It all depends on your personal preferences:
> 
> **Consider these factors:**
> 
> * **Roast Level:**
>     * **Light:** Bright acidity, delicate flavors, often fruity or floral.
>     * **Medium:** Balanced acidity and body, with more pronounced flavors.
>     * **Dark:** Bold, smoky, sometimes bittersweet, with less acidity.
> * **Origin:** Different regions produce beans with unique flavor profiles:
>     * **Latin America:** Often chocolatey, nutty, with medium acidity.
>     * **Africa:** Known for bright acidity, fruity notes, and floral aromas.
>     * **Asia/Pacific:** Earthy, spicy, sometimes with chocolate or caramel notes.
> * **Brewing Method:**  The method you use can significantly impact the flavor.
> 
> **Some Popular and Highly-Regarded Coffees:**
> 
> * **Ethiopian Yirgacheffe:** Known for its floral, citrusy, and tea-like qualities.
> * **Kenyan AA:** Bright acidity, black currant, and berry flavors.
> * **Sumatran Mandheling:** Earthy, spicy, with low acidity.
> * **Colombian Supremo:** Balanced, with chocolate and caramel notes.
> 
> **My Recommendation:**
> 
> * **Start exploring!** Try different roasts, origins, and brewing methods to discover what you enjoy most.
> * **Talk to your local coffee shop:** Baristas are passionate about coffee and can offer great recommendations based on your preferences.
> * **Don't be afraid to experiment:**  The world of coffee is vast and exciting!
> 
> 
> Happy brewing!<end_of_turn>

</font>

In [8]:
message = "what are similar drinks?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>what are similar drinks?</blockquote></font><font size='+1' color='teal'>🤖

> You're looking for drinks that share similarities with coffee! Here are some options, categorized by what aspect of coffee they resemble:
> 
> **Similar Caffeine Kick:**
> 
> * **Tea:**  Comes in endless varieties, with varying caffeine levels. Black tea has the most caffeine, followed by green, white, and herbal teas.
> * **Matcha:** A powdered green tea with a unique earthy flavor and a good amount of caffeine.
> * **Yerba Mate:** A South American beverage with a slightly bitter, grassy flavor and a caffeine boost comparable to coffee.
> 
> **Similar Rich, Roasted Flavor:**
> 
> * **Chicory Coffee:** Made from roasted chicory root, it has a coffee-like flavor but is caffeine-free.
> * **Roasted Barley Tea:**  A caffeine-free option with a nutty, roasted flavor.
> 
> **Similar Creamy, Indulgent Experience:**
> 
> * **Hot Chocolate:**  A classic comfort drink with a rich, chocolatey flavor.
> * **Golden Milk:** A warming drink made with turmeric, ginger, cinnamon, and milk.
> 
> **Similar Ritual and Experience:**
> 
> * **Chai Tea Latte:** A spiced black tea latte with a creamy, comforting flavor.
> * **London Fog:** A blend of Earl Grey tea, steamed milk, and vanilla syrup.
> 
> **Beyond the Usual:**
> 
> * **Kava:** A Polynesian drink made from the kava root, known for its relaxing and social effects.
> * **Guayusa:** A South American tea with a smooth, slightly sweet flavor and a moderate caffeine boost.
> 
> 
> Remember, taste is subjective! Explore these options and see what you enjoy most.<end_of_turn>

</font>

Use the `get_history` method to see how all the context was retained by the Chat class.

In [9]:
print(chat.get_history())

<start_of_turn>user
help me understand how to make coffee<end_of_turn>
<start_of_turn>model
##  Let's Brew Some Coffee! ☕️

Making coffee can be as simple or complex as you like. Here's a breakdown of some popular methods, from beginner-friendly to more advanced:

**1. The Quick & Easy: Instant Coffee**

* **What you need:** Instant coffee granules, hot water
* **How to:** Simply add a spoonful of instant coffee to a mug, pour hot water over it, stir, and enjoy!

**2. The Classic: Drip Coffee Maker**

* **What you need:** Drip coffee maker, ground coffee, filter
* **How to:**
    * Fill the water reservoir with fresh, cold water.
    * Place a filter in the filter basket.
    * Add the desired amount of ground coffee to the filter.
    * Turn on the coffee maker and let it do its thing!

**3. The French Press:**

* **What you need:** French press, coarse ground coffee, hot water
* **How to:**
    * Add coarse ground coffee to the French press.
    * Pour hot water (just off the boil) o